# Day17_0: CNN (합성곱 신경망)

## 📚 학습 목표

**Part 1: 기초**
1. 이미지 데이터의 특성 이해하기
2. 합성곱(Convolution) 연산 이해하기
3. Pooling과 Padding 이해하기
4. CNN 아키텍처 구조 파악하기
5. torchvision으로 이미지 데이터 다루기

**Part 2: 심화**
1. Data Augmentation 적용하기
2. 유명 CNN 아키텍처 (LeNet, VGG, ResNet) 소개
3. Transfer Learning 활용하기
4. CIFAR-10 이미지 분류 실습

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| CNN | 이미지 분류/탐지 | 제품 결함 검사, 의료 영상 분석 |
| Conv Layer | 특징 추출 | 엣지, 질감, 패턴 인식 |
| Pooling | 차원 축소 | 계산 효율화, 위치 불변성 |
| Transfer Learning | 빠른 모델 개발 | 적은 데이터로 고성능 달성 |

**분석가 관점**: CNN은 컴퓨터 비전의 기반 기술입니다. 이미지 분류, 객체 탐지, 세그멘테이션 등 거의 모든 시각 AI 과제에서 CNN 또는 CNN 기반 아키텍처가 사용됩니다!

---

# Part 1: 기초

---

## 1.1 이미지 데이터의 특성

### 이미지는 3D 텐서

이미지 데이터는 (채널, 높이, 너비) 또는 (C, H, W) 형태의 3차원 텐서입니다.

| 이미지 유형 | 채널 수 | 설명 |
|------------|--------|------|
| 흑백 (Grayscale) | 1 | 밝기 정보만 |
| 컬러 (RGB) | 3 | Red, Green, Blue |
| RGBA | 4 | RGB + 투명도 |

```
흑백 이미지: (1, H, W)
컬러 이미지: (3, H, W)
배치 데이터: (N, C, H, W)  # N = 배치 크기
```

In [1]:
import torch
import torch.nn as nn
import numpy as np

# 이미지 텐서 예시
# (배치, 채널, 높이, 너비)
grayscale_image = torch.randn(1, 1, 28, 28)  # 흑백 28x28
color_image = torch.randn(1, 3, 224, 224)    # RGB 224x224

print(f"흑백 이미지 shape: {grayscale_image.shape}")
print(f"컬러 이미지 shape: {color_image.shape}")
print(f"\n흑백 이미지 총 픽셀 수: {grayscale_image.numel()}")
print(f"컬러 이미지 총 픽셀 수: {color_image.numel()}")

흑백 이미지 shape: torch.Size([1, 1, 28, 28])
컬러 이미지 shape: torch.Size([1, 3, 224, 224])

흑백 이미지 총 픽셀 수: 784
컬러 이미지 총 픽셀 수: 150528


### 왜 DNN으로는 이미지 처리가 어려울까?

1. **파라미터 폭발**: 224x224x3 이미지를 펼치면 150,528개 입력! 첫 은닉층 1000개만 해도 1.5억 개 파라미터
2. **공간 정보 손실**: 이미지를 1D로 펼치면 인접 픽셀 관계(엣지, 질감) 정보가 사라짐
3. **위치 의존성**: DNN은 고양이가 왼쪽에 있든 오른쪽에 있든 다른 패턴으로 인식

In [2]:
# DNN으로 이미지 처리 시 파라미터 수 비교
image_size = 224 * 224 * 3  # 컬러 이미지를 1D로 펼친 크기
hidden_neurons = 1000

dnn_params = image_size * hidden_neurons  # 첫 번째 레이어만
print(f"DNN 첫 레이어 파라미터: {dnn_params:,} 개")
print(f"약 {dnn_params / 1e6:.1f}M 파라미터")

DNN 첫 레이어 파라미터: 150,528,000 개
약 150.5M 파라미터


### 이미지 정규화

신경망 학습을 위해 픽셀 값을 정규화합니다.

| 방식 | 범위 | 수식 |
|-----|------|------|
| [0, 1] 정규화 | 0 ~ 1 | pixel / 255 |
| [-1, 1] 정규화 | -1 ~ 1 | (pixel / 255 - 0.5) / 0.5 |
| ImageNet 정규화 | 표준화 | (pixel - mean) / std |

In [3]:
# 정규화 예시
# 0-255 범위의 원본 이미지 (시뮬레이션)
raw_image = torch.randint(0, 256, (3, 4, 4), dtype=torch.float32)
print(f"원본 이미지 (0-255):\n{raw_image[0]}")

# [0, 1] 정규화
normalized_01 = raw_image / 255.0
print(f"\n[0, 1] 정규화:\n{normalized_01[0]}")

# [-1, 1] 정규화
normalized_11 = (raw_image / 255.0 - 0.5) / 0.5
print(f"\n[-1, 1] 정규화:\n{normalized_11[0]}")

원본 이미지 (0-255):
tensor([[ 33.,  63.,  85., 122.],
        [240., 113., 204.,  76.],
        [ 18.,  44., 250., 168.],
        [ 21., 255.,  99., 204.]])

[0, 1] 정규화:
tensor([[0.1294, 0.2471, 0.3333, 0.4784],
        [0.9412, 0.4431, 0.8000, 0.2980],
        [0.0706, 0.1725, 0.9804, 0.6588],
        [0.0824, 1.0000, 0.3882, 0.8000]])

[-1, 1] 정규화:
tensor([[-0.7412, -0.5059, -0.3333, -0.0431],
        [ 0.8824, -0.1137,  0.6000, -0.4039],
        [-0.8588, -0.6549,  0.9608,  0.3176],
        [-0.8353,  1.0000, -0.2235,  0.6000]])


---

## 1.2 합성곱(Convolution) 연산

### 합성곱이란?

**합성곱**: 작은 필터(커널)를 이미지 위에서 슬라이딩하며 요소별 곱의 합을 계산

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdna%2FcitQ3T%2FbtsmlifnSA3%2FAAAAAAAAAAAAAAAAAAAAALywDw66YaR6woaoy-blQeVi_wt0l6BnMJ3yfPVDGh3X%2Fimg.jpg%3Fcredential%3DyqXZFxpELC7KVnFOS48ylbz2pIh7yKj8%26expires%3D1769871599%26allow_ip%3D%26allow_referer%3D%26signature%3DG0Gb4Xc%252FkZ%252F6tTyfE03JudrjbFw%253D">

```
입력 이미지    필터(커널)       특징 맵
[1 2 3 4]     [1 0]          [? ? ?]
[5 6 7 8]  *  [0 1]    =     [? ? ?]
[9 0 1 2]                    [? ? ?]
[3 4 5 6]
```

### 핵심 개념

| 개념 | 설명 | 비유 |
|-----|------|------|
| 필터/커널 | 특정 패턴을 감지하는 가중치 행렬 | 돋보기 |
| 특징 맵 | 필터 적용 결과, 특징의 위치와 강도 | 지도 |
| 수용 영역 | 하나의 출력 뉴런이 보는 입력 영역 | 시야 |

In [4]:
# 간단한 합성곱 연산 직접 구현
import torch.nn.functional as F

# 5x5 입력 이미지 (1채널)
image = torch.tensor([
    [1, 2, 3, 0, 1],
    [0, 1, 2, 3, 2],
    [1, 0, 1, 0, 1],
    [2, 1, 0, 1, 0],
    [0, 1, 2, 1, 1]
], dtype=torch.float32)

# 3x3 필터 (수직 엣지 검출)
vertical_edge_filter = torch.tensor([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1]
], dtype=torch.float32)

print(f"입력 이미지:\n{image}")
print(f"\n수직 엣지 필터:\n{vertical_edge_filter}")

입력 이미지:
tensor([[1., 2., 3., 0., 1.],
        [0., 1., 2., 3., 2.],
        [1., 0., 1., 0., 1.],
        [2., 1., 0., 1., 0.],
        [0., 1., 2., 1., 1.]])

수직 엣지 필터:
tensor([[-1.,  0.,  1.],
        [-1.,  0.,  1.],
        [-1.,  0.,  1.]])


In [ ]:
# Conv2d로 합성곱 수행
# (배치, 채널, 높이, 너비) 형태로 변환
image_batch = image.unsqueeze(0).unsqueeze(0)  # (1, 1, 5, 5)
filter_batch = vertical_edge_filter.unsqueeze(0).unsqueeze(0)  # (1, 1, 3, 3)

# F.conv2d로 직접 합성곱
output = F.conv2d(image_batch, filter_batch)
print(f"합성곱 결과 shape: {output.shape}")
print(f"\n특징 맵:\n{output.squeeze()}")

합성곱 결과 shape: torch.Size([1, 1, 3, 3])

특징 맵:
tensor([[ 4.,  0., -2.],
        [ 0.,  2.,  0.],
        [ 0.,  0., -1.]])


### nn.Conv2d 사용하기

```python
nn.Conv2d(
    in_channels,   # 입력 채널 수 (RGB면 3)
    out_channels,  # 출력 채널 수 = 필터 개수
    kernel_size,   # 필터 크기 (3이면 3x3)
    stride=1,      # 필터 이동 간격
    padding=0      # 입력 가장자리 패딩
)
```

In [6]:
# nn.Conv2d 예시
# 입력: 3채널(RGB), 출력: 16개 특징 맵, 3x3 커널
conv = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)

# 더미 입력: (배치=1, 채널=3, 높이=32, 너비=32)
dummy_input = torch.randn(1, 3, 32, 32)

# 합성곱 수행
output = conv(dummy_input)

print(f"입력 shape: {dummy_input.shape}")
print(f"출력 shape: {output.shape}")
print(f"\n출력 크기가 30x30인 이유: 32 - 3 + 1 = 30 (패딩 없이 3x3 커널)")

입력 shape: torch.Size([1, 3, 32, 32])
출력 shape: torch.Size([1, 16, 30, 30])

출력 크기가 30x30인 이유: 32 - 3 + 1 = 30 (패딩 없이 3x3 커널)


### 💡 실무 예시: 다양한 필터로 특징 추출

CNN은 학습을 통해 최적의 필터를 자동으로 찾습니다!

In [7]:
# 여러 종류의 엣지 검출 필터
# 수직 엣지
vertical = torch.tensor([[-1, 0, 1],
                         [-1, 0, 1],
                         [-1, 0, 1]], dtype=torch.float32)

# 수평 엣지
horizontal = torch.tensor([[-1, -1, -1],
                           [ 0,  0,  0],
                           [ 1,  1,  1]], dtype=torch.float32)

# Sobel 필터 (엣지 강조)
sobel_x = torch.tensor([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]], dtype=torch.float32)

print("수직 엣지 필터:")
print(vertical)
print("\n수평 엣지 필터:")
print(horizontal)
print("\nSobel X 필터:")
print(sobel_x)

수직 엣지 필터:
tensor([[-1.,  0.,  1.],
        [-1.,  0.,  1.],
        [-1.,  0.,  1.]])

수평 엣지 필터:
tensor([[-1., -1., -1.],
        [ 0.,  0.,  0.],
        [ 1.,  1.,  1.]])

Sobel X 필터:
tensor([[-1.,  0.,  1.],
        [-2.,  0.,  2.],
        [-1.,  0.,  1.]])


---

## 1.3 Padding과 Stride

### Stride (보폭)

필터가 이동하는 간격

| Stride | 효과 |
|--------|------|
| 1 | 출력 크기 유지에 가까움 |
| 2 | 출력 크기 약 절반 |
| n | 출력 크기 약 1/n |

### Padding (패딩)

입력 이미지 가장자리에 값(보통 0)을 추가

| Padding | 효과 |
|---------|------|
| 0 | 출력 크기 감소 |
| 'same' | 출력 크기 = 입력 크기 |
| (k-1)/2 | 3x3 커널이면 padding=1 |

In [8]:
# Stride와 Padding 효과 비교
dummy_input = torch.randn(1, 3, 32, 32)

# 기본 (stride=1, padding=0)
conv_default = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=0)
out_default = conv_default(dummy_input)
print(f"기본 (s=1, p=0): {dummy_input.shape} -> {out_default.shape}")


기본 (s=1, p=0): torch.Size([1, 3, 32, 32]) -> torch.Size([1, 16, 30, 30])


In [9]:

# padding=1 (출력 크기 유지)
conv_padded = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
out_padded = conv_padded(dummy_input)
print(f"패딩 (s=1, p=1): {dummy_input.shape} -> {out_padded.shape}")


패딩 (s=1, p=1): torch.Size([1, 3, 32, 32]) -> torch.Size([1, 16, 32, 32])


In [10]:

# stride=2 (다운샘플링)
conv_stride = nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1)
out_stride = conv_stride(dummy_input)
print(f"스트라이드 (s=2, p=1): {dummy_input.shape} -> {out_stride.shape}")

스트라이드 (s=2, p=1): torch.Size([1, 3, 32, 32]) -> torch.Size([1, 16, 16, 16])


### 출력 크기 계산 공식

```
출력 크기 = floor((입력 크기 + 2*padding - kernel_size) / stride) + 1
```

In [11]:
def calc_output_size(input_size, kernel_size, stride=1, padding=0):
    """합성곱 출력 크기 계산"""
    return (input_size + 2 * padding - kernel_size) // stride + 1

# 예시 계산
examples = [
    (32, 3, 1, 0),   # 기본
    (32, 3, 1, 1),   # 패딩으로 크기 유지
    (32, 3, 2, 1),   # 다운샘플링
    (224, 7, 2, 3),  # ResNet 첫 레이어
]

print("출력 크기 계산 예시:")
print("="*50)
for inp, k, s, p in examples:
    out = calc_output_size(inp, k, s, p)
    print(f"입력={inp}, 커널={k}x{k}, stride={s}, padding={p} -> 출력={out}")

출력 크기 계산 예시:
입력=32, 커널=3x3, stride=1, padding=0 -> 출력=30
입력=32, 커널=3x3, stride=1, padding=1 -> 출력=32
입력=32, 커널=3x3, stride=2, padding=1 -> 출력=16
입력=224, 커널=7x7, stride=2, padding=3 -> 출력=112


---

## 1.4 Pooling (풀링)

### Pooling이란?

**풀링**: 특징 맵의 크기를 줄이는 다운샘플링 연산

| 종류 | 연산 | 특징 |
|-----|------|------|
| Max Pooling | 영역 내 최댓값 | 가장 강한 활성화 보존 |
| Average Pooling | 영역 내 평균 | 부드러운 다운샘플링 |
| Global Average Pooling | 전체 평균 | FC 레이어 대체 가능 |

### Pooling의 장점

1. **계산량 감소**: 파라미터 없이 차원 축소
2. **과적합 방지**: 불필요한 세부 정보 제거
3. **위치 불변성**: 특징의 정확한 위치보다 존재 여부 중요

In [12]:
# Max Pooling vs Average Pooling
feature_map = torch.tensor([
    [[1, 2, 3, 4],
     [5, 6, 7, 8],
     [9, 0, 1, 2],
     [3, 4, 5, 6]]
], dtype=torch.float32).unsqueeze(0)  # (1, 1, 4, 4)

print(f"입력 특징 맵:\n{feature_map.squeeze()}")


입력 특징 맵:
tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.],
        [9., 0., 1., 2.],
        [3., 4., 5., 6.]])


In [13]:

# 2x2 Max Pooling
max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
max_pooled = max_pool(feature_map)
print(f"\n2x2 Max Pooling:\n{max_pooled.squeeze()}")



2x2 Max Pooling:
tensor([[6., 8.],
        [9., 6.]])


In [14]:

# 2x2 Average Pooling
avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
avg_pooled = avg_pool(feature_map)
print(f"\n2x2 Average Pooling:\n{avg_pooled.squeeze()}")


2x2 Average Pooling:
tensor([[3.5000, 5.5000],
        [4.0000, 3.5000]])


In [15]:
# Global Average Pooling (GAP)
# 각 채널의 전체 평균 -> (N, C, H, W) -> (N, C, 1, 1)
feature_maps = torch.randn(1, 16, 7, 7)  # 16개 채널의 7x7 특징 맵

gap = nn.AdaptiveAvgPool2d(1)  # 출력 크기를 1x1로
gap_output = gap(feature_maps)

print(f"입력: {feature_maps.shape}")
print(f"GAP 출력: {gap_output.shape}")
print(f"\n이렇게 하면 FC 레이어 없이 바로 분류 가능!")

입력: torch.Size([1, 16, 7, 7])
GAP 출력: torch.Size([1, 16, 1, 1])

이렇게 하면 FC 레이어 없이 바로 분류 가능!


---

## 1.4 경사 하강법 (Gradient Descent)

### 직관적 이해: 언덕 내려가기

```
목표: 손실 함수의 최솟값 찾기
방법: 현재 위치에서 가장 가파르게 내려가는 방향으로 이동

새 위치 = 현재 위치 - 학습률 * 기울기
w_new = w_old - lr * dL/dw
```

---

## 1.5 CNN 아키텍처 구조

### 전형적인 CNN 구조

```mermaid
 graph LR
     A["입력 이미지"] 
     B1[Conv]
     B2[ReLU]
     B3[Pool]
     C1[Conv]
     C2[ReLU]
     C3[Pool]
     D[Flatten]
     E1[FC]
     E2[ReLU]
     F["출력 (클래스별 점수)"]
 
     A --> B1 --> B2 --> B3 --> C1 --> C2 --> C3 --> D --> E1 --> E2 --> F
 
     subgraph 특징 추출부
         B1 --> B2 --> B3
         B3 --> C1 --> C2 --> C3
     end
 
     subgraph 분류부
         D --> E1 --> E2
     end
```


### CNN의 핵심 아이디어

1. **지역 연결성**: 각 뉴런이 입력의 일부만 봄 (수용 영역)
2. **파라미터 공유**: 같은 필터를 전체 이미지에 적용
3. **계층적 학습**: 저수준(엣지) → 고수준(얼굴) 특징

In [ ]:
# 간단한 CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        # 특징 추출부 (Convolutional Layers)
        self.features = nn.Sequential(
            # Conv Block 1: (3, 32, 32) -> (32, 16, 16)
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # Conv Block 2: (32, 16, 16) -> (64, 8, 8)
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # Conv Block 3: (64, 8, 8) -> (128, 4, 4)
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        # 분류부 (Fully Connected Layers)
        self.classifier = nn.Sequential(
            nn.Flatten(),  # (128, 4, 4) -> 2048
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# 모델 생성 및 테스트
model = SimpleCNN(num_classes=10)
print(model)

In [ ]:
# 순전파 테스트
dummy_input = torch.randn(4, 3, 32, 32)  # 배치 4개, RGB, 32x32
output = model(dummy_input)

print(f"입력 shape: {dummy_input.shape}")
print(f"출력 shape: {output.shape}")

# 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())
print(f"\n총 파라미터 수: {total_params:,}")

---

## 1.6 torchvision으로 이미지 다루기

### torchvision 구성요소

| 모듈 | 설명 |
|-----|------|
| `datasets` | 유명 데이터셋 (MNIST, CIFAR, ImageNet 등) |
| `transforms` | 이미지 전처리/증강 |
| `models` | 사전 학습 모델 (ResNet, VGG 등) |

In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 기본 전처리 파이프라인
transform = transforms.Compose([
    transforms.ToTensor(),  # PIL Image -> Tensor (0-255 -> 0-1)
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # [-1, 1] 정규화
])

print("transforms.Compose 파이프라인:")
print(transform)

In [ ]:
# CIFAR-10 데이터셋 로드
# 다운로드 경로 설정
data_path = './datasets'

# 훈련 데이터
train_dataset = torchvision.datasets.CIFAR10(
    root=data_path,
    train=True,
    download=True,
    transform=transform
)

# 테스트 데이터
test_dataset = torchvision.datasets.CIFAR10(
    root=data_path,
    train=False,
    download=True,
    transform=transform
)

print(f"훈련 데이터: {len(train_dataset)} 샘플")
print(f"테스트 데이터: {len(test_dataset)} 샘플")

In [ ]:
# CIFAR-10 클래스
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# 데이터 확인
image, label = train_dataset[0]
print(f"이미지 shape: {image.shape}")
print(f"레이블: {label} ({classes[label]})")
print(f"픽셀 값 범위: [{image.min():.2f}, {image.max():.2f}]")

In [ ]:
# DataLoader 생성
batch_size = 64

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0  # 멀티프로세싱
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)

print(f"훈련 배치 수: {len(train_loader)}")
print(f"테스트 배치 수: {len(test_loader)}")

In [ ]:
# 샘플 이미지 시각화
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 첫 배치 가져오기
images, labels = next(iter(train_loader))

# 이미지 역정규화 (시각화용)
def denormalize(tensor):
    return tensor * 0.5 + 0.5

# 처음 16개 이미지 표시
fig = make_subplots(rows=2, cols=8, subplot_titles=[classes[labels[i]] for i in range(16)])

for i in range(16):
    img = denormalize(images[i]).permute(1, 2, 0).numpy()  # (C,H,W) -> (H,W,C)
    row = i // 8 + 1
    col = i % 8 + 1
    fig.add_trace(go.Image(z=(img * 255).astype(np.uint8)), row=row, col=col)

fig.update_layout(title='CIFAR-10 샘플 이미지', height=300, showlegend=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

---

# Part 2: 심화

---

## 2.1 Data Augmentation (데이터 증강)

### 왜 데이터 증강이 필요한가?

1. **데이터 부족 해결**: 적은 데이터로 더 많은 학습
2. **과적합 방지**: 다양한 변형에 강건한 모델
3. **일반화 성능 향상**: 실제 환경의 다양성 반영

### 주요 증강 기법

| 기법 | 설명 | transforms 함수 |
|-----|------|-----------------|
| 수평 뒤집기 | 좌우 반전 | RandomHorizontalFlip |
| 회전 | 각도 회전 | RandomRotation |
| 크롭 | 무작위 자르기 | RandomCrop, RandomResizedCrop |
| 색상 변환 | 밝기/대비/채도 | ColorJitter |

In [ ]:
# Data Augmentation 파이프라인
train_transform = transforms.Compose([
    # 무작위 자르기 후 리사이즈
    transforms.RandomCrop(32, padding=4),
    # 50% 확률로 좌우 반전
    transforms.RandomHorizontalFlip(),
    # 색상 변환
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    # 텐서 변환
    transforms.ToTensor(),
    # 정규화
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 테스트용 (증강 없음)
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

print("훈련 데이터 증강 파이프라인:")
print(train_transform)

In [ ]:
# 증강 효과 시각화
from PIL import Image

# 원본 이미지 가져오기 (증강 전)
simple_transform = transforms.Compose([transforms.ToTensor()])
simple_dataset = torchvision.datasets.CIFAR10(
    root=data_path, train=True, download=False, transform=simple_transform
)
original_img, label = simple_dataset[0]

# PIL Image로 변환 (증강 적용을 위해)
pil_img = transforms.ToPILImage()(original_img)

# 다양한 증강 적용
augmentations = [
    ('원본', transforms.ToTensor()),
    ('수평 뒤집기', transforms.Compose([transforms.RandomHorizontalFlip(p=1.0), transforms.ToTensor()])),
    ('회전', transforms.Compose([transforms.RandomRotation(30), transforms.ToTensor()])),
    ('색상 변환', transforms.Compose([transforms.ColorJitter(brightness=0.5), transforms.ToTensor()])),
]

fig = make_subplots(rows=1, cols=4, subplot_titles=[name for name, _ in augmentations])

for i, (name, aug_transform) in enumerate(augmentations):
    aug_img = aug_transform(pil_img).permute(1, 2, 0).numpy()
    fig.add_trace(go.Image(z=(aug_img * 255).astype(np.uint8)), row=1, col=i+1)

fig.update_layout(title=f'Data Augmentation 효과 - {classes[label]}', height=200)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

---

## 2.2 유명 CNN 아키텍처

### CNN 발전 역사

| 모델 | 년도 | 특징 | 파라미터 |
|-----|------|------|----------|
| LeNet-5 | 1998 | 최초의 성공적 CNN | 60K |
| AlexNet | 2012 | 딥러닝 부흥 시작 | 60M |
| VGG16 | 2014 | 깊고 단순한 구조 | 138M |
| ResNet | 2015 | Skip Connection | 25M (ResNet-50) |

In [ ]:
# LeNet-5 스타일 모델 (간단한 구조)
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),    # (3, 32, 32) -> (6, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                # (6, 14, 14)
            nn.Conv2d(6, 16, kernel_size=5),   # (16, 10, 10)
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                 # (16, 5, 5)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

lenet = LeNet()
print(f"LeNet 파라미터: {sum(p.numel() for p in lenet.parameters()):,}")

In [ ]:
# VGG 스타일 블록 (3x3 Conv 반복)
def make_vgg_block(in_channels, out_channels, num_convs):
    """VGG 스타일 합성곱 블록"""
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(2, 2))
    return nn.Sequential(*layers)

# 간단한 VGG 스타일 모델
class MiniVGG(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            make_vgg_block(3, 64, 2),      # (64, 16, 16)
            make_vgg_block(64, 128, 2),    # (128, 8, 8)
            make_vgg_block(128, 256, 2),   # (256, 4, 4)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

vgg = MiniVGG()
print(f"MiniVGG 파라미터: {sum(p.numel() for p in vgg.parameters()):,}")

### ResNet의 혁신: Skip Connection (잔차 연결)

```
기존:  x -> [Conv -> ReLU -> Conv] -> F(x)
ResNet: x -> [Conv -> ReLU -> Conv] + x -> F(x) + x
```

Skip Connection의 장점:
- 기울기 소실 문제 완화
- 매우 깊은 네트워크 학습 가능 (100+ layers)

In [ ]:
# 간단한 Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Skip connection (차원이 다를 경우 조정)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # Skip Connection!
        out = F.relu(out)
        return out

# 테스트
block = ResidualBlock(64, 128, stride=2)
x = torch.randn(1, 64, 16, 16)
out = block(x)
print(f"입력: {x.shape} -> 출력: {out.shape}")

---

## 2.3 Transfer Learning (전이 학습)

### 전이 학습이란?

**전이 학습**: 대규모 데이터셋(ImageNet)으로 사전 학습된 모델을 가져와 새로운 과제에 적용

### 두 가지 전략

| 전략 | 설명 | 언제 사용? |
|-----|------|----------|
| Feature Extraction | 사전 학습 가중치 고정, 마지막 레이어만 학습 | 데이터가 매우 적을 때 |
| Fine-tuning | 전체 또는 일부 레이어 재학습 | 데이터가 어느 정도 있을 때 |

In [ ]:
import torchvision.models as models

# 사전 학습된 ResNet18 불러오기
resnet18 = models.resnet18(weights='IMAGENET1K_V1')

print("ResNet18 마지막 레이어:")
print(resnet18.fc)

In [ ]:
# Feature Extraction: 사전 학습 가중치 고정
# 마지막 FC 레이어만 새로 정의

# 1. 모든 파라미터 동결
for param in resnet18.parameters():
    param.requires_grad = False

# 2. 마지막 FC 레이어 교체 (10개 클래스용)
num_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_features, 10)

print(f"새 FC 레이어 입력 크기: {num_features}")
print(f"학습 가능한 파라미터: {sum(p.numel() for p in resnet18.parameters() if p.requires_grad):,}")

In [ ]:
# Fine-tuning: 일부 레이어 동결 해제
resnet18_ft = models.resnet18(weights='IMAGENET1K_V1')

# 1. 모든 파라미터 동결
for param in resnet18_ft.parameters():
    param.requires_grad = False

# 2. 마지막 레이어들 동결 해제 (layer4 + fc)
for param in resnet18_ft.layer4.parameters():
    param.requires_grad = True

# 3. FC 레이어 교체
resnet18_ft.fc = nn.Linear(resnet18_ft.fc.in_features, 10)

trainable_params = sum(p.numel() for p in resnet18_ft.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in resnet18_ft.parameters())

print(f"전체 파라미터: {total_params:,}")
print(f"학습 가능 파라미터: {trainable_params:,}")
print(f"학습 비율: {trainable_params/total_params:.1%}")

### 💡 실무 팁: ImageNet 정규화

사전 학습 모델 사용 시 반드시 동일한 정규화 적용!

In [ ]:
# ImageNet 정규화 값
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

# 전이 학습용 전처리
transfer_transform = transforms.Compose([
    transforms.Resize(224),  # ResNet 입력 크기
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

print("ImageNet 정규화 값:")
print(f"Mean: {imagenet_mean}")
print(f"Std: {imagenet_std}")

---

## 2.4 CIFAR-10 분류 실습

### 전체 학습 파이프라인

1. 데이터 준비 (증강 포함)
2. 모델 정의
3. 학습 루프
4. 평가 및 시각화

In [ ]:
# 1. 데이터 준비
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))  # CIFAR-10 통계
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

# 데이터셋
train_dataset = torchvision.datasets.CIFAR10(
    root=data_path, train=True, download=True, transform=train_transform
)
test_dataset = torchvision.datasets.CIFAR10(
    root=data_path, train=False, download=True, transform=test_transform
)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=0)

print(f"훈련 배치: {len(train_loader)}, 테스트 배치: {len(test_loader)}")

In [ ]:
# 2. 모델 정의 (SimpleCNN 사용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 device: {device}")

model = SimpleCNN(num_classes=10).to(device)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [ ]:
# 3. 학습 함수
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

In [ ]:
# 4. 학습 실행 (5 에포크만 - 시간 절약)
epochs = 5
history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': []}

print("학습 시작!")
print("="*60)

for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    scheduler.step()
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['test_loss'].append(test_loss)
    history['test_acc'].append(test_acc)
    
    print(f"Epoch {epoch+1}/{epochs}: "
          f"Train Loss={train_loss:.4f}, Train Acc={train_acc:.2f}%, "
          f"Test Loss={test_loss:.4f}, Test Acc={test_acc:.2f}%")

print("="*60)
print("학습 완료!")

In [ ]:
# 5. 학습 곡선 시각화
import pandas as pd

history_df = pd.DataFrame(history)
history_df['epoch'] = range(1, epochs + 1)

# 손실 곡선
fig = make_subplots(rows=1, cols=2, subplot_titles=['Loss', 'Accuracy'])

fig.add_trace(go.Scatter(x=history_df['epoch'], y=history_df['train_loss'], name='Train Loss'), row=1, col=1)
fig.add_trace(go.Scatter(x=history_df['epoch'], y=history_df['test_loss'], name='Test Loss'), row=1, col=1)

fig.add_trace(go.Scatter(x=history_df['epoch'], y=history_df['train_acc'], name='Train Acc'), row=1, col=2)
fig.add_trace(go.Scatter(x=history_df['epoch'], y=history_df['test_acc'], name='Test Acc'), row=1, col=2)

fig.update_layout(title='학습 곡선', height=400)
fig.update_xaxes(title_text='Epoch')
fig.show()

In [ ]:
# 6. 혼동 행렬 (Confusion Matrix)
from sklearn.metrics import confusion_matrix

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

cm = confusion_matrix(all_labels, all_preds)

# 시각화
fig = px.imshow(cm, 
                labels=dict(x="예측", y="실제", color="개수"),
                x=classes, y=classes,
                color_continuous_scale='Blues')
fig.update_layout(title='혼동 행렬 (Confusion Matrix)')
fig.show()

In [ ]:
# 7. 예측 샘플 시각화
model.eval()
images, labels = next(iter(test_loader))
images = images.to(device)

with torch.no_grad():
    outputs = model(images)
    _, predicted = outputs.max(1)

# 처음 8개 샘플
fig = make_subplots(rows=2, cols=4)

for i in range(8):
    img = images[i].cpu()
    # 역정규화
    img = img * torch.tensor([0.2470, 0.2435, 0.2616]).view(3, 1, 1) + \
          torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
    img = img.permute(1, 2, 0).numpy()
    img = np.clip(img, 0, 1)
    
    row = i // 4 + 1
    col = i % 4 + 1
    fig.add_trace(go.Image(z=(img * 255).astype(np.uint8)), row=row, col=col)
    
    label = classes[labels[i]]
    pred = classes[predicted[i]]
    color = 'green' if label == pred else 'red'
    fig.add_annotation(x=0.5, y=-0.1, text=f"{pred}", 
                       xref=f"x{i+1} domain", yref=f"y{i+1} domain",
                       showarrow=False, font=dict(color=color))

fig.update_layout(title='예측 결과 (초록=정답, 빨강=오답)', height=300)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. Conv2d 출력 크기 계산 ⭐

**문제**: 64x64 크기의 RGB 이미지에 다음 Conv2d를 적용했을 때 출력 크기를 계산하세요.

```python
nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2)
```

In [ ]:
# 여기에 코드를 작성하세요


### Q2. Max Pooling 효과 ⭐

**문제**: 4x4 특징 맵에 2x2 Max Pooling (stride=2)을 적용한 결과를 출력하세요.

```python
feature_map = torch.tensor([[1, 3, 2, 4],
                            [5, 6, 7, 8],
                            [3, 2, 1, 0],
                            [9, 8, 7, 6]], dtype=torch.float32)
```

In [ ]:
# 여기에 코드를 작성하세요


### Q3. 이미지 정규화 ⭐⭐

**문제**: 0-255 범위의 이미지를 [-1, 1] 범위로 정규화하는 코드를 작성하세요.

In [ ]:
# 0-255 범위 이미지 (시뮬레이션)
raw_image = torch.randint(0, 256, (3, 32, 32), dtype=torch.float32)

# 여기에 코드를 작성하세요


### Q4. CNN 아키텍처 분석 ⭐⭐

**문제**: 다음 CNN 모델의 각 레이어 출력 shape를 계산하세요.

입력: (1, 3, 32, 32)

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3, 16, 3, padding=1),    # ?
    nn.ReLU(),
    nn.MaxPool2d(2, 2),                 # ?
    nn.Conv2d(16, 32, 3, padding=1),   # ?
    nn.ReLU(),
    nn.MaxPool2d(2, 2),                 # ?
    nn.Flatten(),                       # ?
)

# 여기에 코드를 작성하세요


### Q5. Data Augmentation 적용 ⭐⭐

**문제**: 다음 증강 기법을 포함하는 transform 파이프라인을 작성하세요.

1. 무작위 크롭 (32x32, padding=4)
2. 무작위 수평 뒤집기
3. 무작위 회전 (최대 15도)
4. 텐서 변환
5. 정규화 (mean=0.5, std=0.5)

In [ ]:
# 여기에 코드를 작성하세요


### Q6. torchvision 데이터셋 로드 ⭐⭐

**문제**: MNIST 데이터셋을 로드하고, 첫 번째 이미지의 shape와 레이블을 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q7. Transfer Learning 설정 ⭐⭐⭐

**문제**: 사전 학습된 ResNet18을 불러와서 5개 클래스 분류 문제에 적용하세요.

요구사항:
1. 모든 레이어 동결
2. 마지막 FC 레이어만 5개 클래스용으로 교체
3. 학습 가능한 파라미터 수 출력

In [ ]:
# 여기에 코드를 작성하세요


### Q8. Fine-tuning 구현 ⭐⭐⭐

**문제**: ResNet18에서 layer4와 fc 레이어만 학습 가능하도록 설정하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 커스텀 CNN 모델 ⭐⭐⭐⭐

**문제**: 다음 조건을 만족하는 CNN 모델을 정의하세요.

- 입력: 1채널 (흑백), 28x28
- Conv 블록 3개 (채널: 32 -> 64 -> 128)
- 각 블록: Conv -> BatchNorm -> ReLU -> MaxPool
- Global Average Pooling
- 출력: 10개 클래스

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 전체 학습 파이프라인 ⭐⭐⭐⭐⭐

**문제**: CIFAR-10 데이터셋으로 CNN을 학습하는 전체 파이프라인을 구현하세요.

요구사항:
1. Data Augmentation 적용 (훈련 데이터)
2. 커스텀 CNN 모델 사용
3. 3 에포크 학습
4. 학습/테스트 손실 및 정확도 기록
5. Plotly로 학습 곡선 시각화

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 함수 | 실무 활용 |
|-----|----------|----------|
| 이미지 텐서 | (N, C, H, W) | 배치 처리 |
| Conv2d | nn.Conv2d(in, out, kernel) | 특징 추출 |
| Pooling | nn.MaxPool2d, nn.AvgPool2d | 다운샘플링 |
| Padding/Stride | padding=1, stride=2 | 출력 크기 조절 |
| torchvision | datasets, transforms | 데이터 전처리 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 메서드 | 언제 사용? |
|-----|-----------|----------|
| Data Augmentation | RandomCrop, RandomHorizontalFlip | 과적합 방지 |
| LeNet/VGG/ResNet | torchvision.models | 검증된 아키텍처 |
| Feature Extraction | requires_grad=False | 데이터 적을 때 |
| Fine-tuning | 일부 레이어 해제 | 데이터 충분할 때 |

### CNN 설계 핵심 패턴

```python
# 특징 추출부
for i in range(num_blocks):
    Conv2d -> BatchNorm -> ReLU -> MaxPool
    (채널 수 증가, 공간 크기 감소)

# 분류부
Flatten / GlobalAvgPool
-> Linear -> ReLU -> Dropout -> Linear
```

### 💡 실무 팁

1. **데이터가 적으면**: Transfer Learning 필수
2. **과적합 문제**: Data Augmentation + Dropout
3. **학습이 불안정하면**: BatchNorm 추가
4. **기울기 소실**: Skip Connection (ResNet 스타일)
5. **ImageNet 모델 사용 시**: 반드시 동일 정규화 적용